In [1]:
import streamlit as st
import alpaca_trade_api as aplpaca
from alpaca_trade_api.rest import REST, TimeFrame
import os
import time
from dotenv import load_dotenv

from web3 import Web3
from pathlib import Path

import json
from io import StringIO

load_dotenv()

2022-07-31 20:29:02.684 INFO    numexpr.utils: Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2022-07-31 20:29:02.685 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


True

In [2]:
class Grader:
    # add event filters to Grader class 
    def __init__(self, trader_id, log_address):
        self.trader_id = trader_id
        self.log_address = log_address
        
    def load_contract(self):
        contract_address = self.log_address
        with open(Path('logger.json')) as f:
            logger_abi = json.load(f)
            
            self.contract = w3.eth.contract(
            address=contract_address,
            abi=logger_abi)
            
            self.new_trade_filter = self.contract.events.newTrade.createFilter(fromBlock="latest")
            
        return None

In [3]:

w3 = Web3(Web3.HTTPProvider(os.getenv("WEB3_PROVIDER_URI")))

def load_deployer_contract():
    contract_address = os.getenv("DEPLOYER_ADDRESS") 
    with open(Path('deployer.json')) as f:
        deployer_abi = json.load(f)

    deployer_contract = w3.eth.contract(
        address=contract_address,
        abi=deployer_abi
        )
    trader_dict = {}
    return deployer_contract, trader_dict

def load_contract_new(log_addr):
    contract_address = log_addr
    with open(Path('logger.json')) as f:
            logger_abi = json.load(f)

    logger_contract = w3.eth.contract(
        address=contract_address,
        abi=logger_abi
            )
    return logger_contract

def handle_new_log(event):
    io = StringIO((Web3.toJSON(event)))
    return json.load(io)
    

def handle_new_trade(event):
    io = StringIO((Web3.toJSON(event)))
    return json.load(io)

In [4]:
deployer, trader_dict = load_deployer_contract()

deployer.address

'0xfC03E39e8f77eEa074cC2Dd34a4FaC23E6eC3d39'

In [6]:
# Run script below. Deploy DEPLOYER contract and hit createLog.
# Copy newLogAddress to to At Address, TraderId to _TRADERID, and DEPLOYER address to _SERVER to deploy LOGGER contract.

# Create a filter to find new log contracts
new_log_filter = deployer.events.logCreated.createFilter(fromBlock="latest")

# Listen
print("Starting while loop...")
while True:
    # 1) Listen for new traders creating trading logs
    new_log_event = handle_new_log(new_log_filter.get_new_entries())
    if new_log_event:
        print("--------------------------")
        print((new_log_event[0]['args']))
        print("--------------------------")
        
        # add the information to a Grader object stored at that trader's ID variable
        new_log = new_log_event[0]['args']
        trader_dict[new_log['TraderId']]=Grader(new_log['TraderId'], new_log['newLogAddress'])
        trader_dict[new_log['TraderId']].load_contract()
            
    
    # loop through each trader and look for new trades at their log's contract address
    for key in trader_dict.keys():
        new_trade_event = handle_new_trade(trader_dict[key].new_trade_filter.get_new_entries())
        if new_trade_event:
            print("--------------------------")
            print((new_trade_event))
            print("--------------------------")
            
            
    time.sleep(3)

Starting while loop...
--------------------------
{'TraderId': 2, 'newLogAddress': '0x961Da170614D6f0C259a66a0C7f1a8b51A82cdaD'}
--------------------------
--------------------------
{'TraderId': 3, 'newLogAddress': '0x1A15C322223255Bd5C8d3471BCE5C8f3043fD03C'}
--------------------------
--------------------------
[{'args': {'TraderId': 2, 'inputTraderAddress': '0x53605aA1FE094C90bF3d6ad7DaFB047f4970d71B', 'tradeNum': 0, 'inputOpen': True, 'inputSymbol': "'TSLA'", 'inputSize': "'100'", 'inputEntry': "'895'", 'inputOptionsData': "'900 Call'"}, 'event': 'newTrade', 'logIndex': 0, 'transactionIndex': 0, 'transactionHash': '0x27aa10318d09e52e6645ad087defe83bd094366a4d52f27d6e4119bdfd3e0076', 'address': '0x961Da170614D6f0C259a66a0C7f1a8b51A82cdaD', 'blockHash': '0x101fb812a626fb90c45f3b76cb644c14f9c56b1a5acec72a51527c42dbdc4698', 'blockNumber': 22}]
--------------------------


KeyboardInterrupt: 